Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In addition, we import clinical decision files and use it to "override" decisions that were made by analyzing the ROI for specialty & cpt code combination.

Finally, several outputs are prepared for reporting purposes, including the calculation of a projected AA approval rate.

In [ ]:
## Import required packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyodbc
import os

## Import Data

#### Import data directly from SQL databases

Use referrals.sql, claims.sql, referrals_new.sql, cpt_desc.sql (saved in the same folder as this script) to pull data from relevants servers and databases.

In [ ]:
with open('referrals.sql', 'r') as myfile:
    cpts_sql_str=myfile.read().replace('\n', ' ')

cnxn_cpts = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

cpts = pd.read_sql(cpts_sql_str, cnxn_cpts)

cnxn_cpts.close()

In [ ]:
with open('claims.sql', 'r') as myfile:
    claims_sql_str=myfile.read().replace('\n', ' ')

cnxn_claims = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=NATIONAL_ANALYTICS')

claims = pd.read_sql(claims_sql_str, cnxn_claims)

cnxn_claims.close()

In the next step you will import the current version of the clinical decisions.  Note that this is an "active" or "living" document that is kept up-to-date week-by-week.  As such, you should retrieve the most up-to-date version from the link below before running the next line of code:
\\is_file\datashare\Auto-Approval Management

In [ ]:
clinical_decisions_current = pd.read_excel('../data/AADictionary_Master.xlsx', sheet_name='Specialty Summary')

In [ ]:
clinical_decisions_current = clinical_decisions_current[['Specialty', 'CPT_Code', 'is_PPL', 'Decision_Source',
       'Decision_Maker', 'Reason', 'Decision_Date', 'Decision']]

## Clean Referrals

A list of all specialties is used repeatedly in the code to loop through specialties and perform certain tasks.

In [ ]:
# A list of all specialties is used repeatedly in the code to loop through 
# specialties and perform certain tasks.
list_o_specs = cpts['Specialty'].unique().tolist()

In [ ]:
## Update date fields to conform to python datetime
cpts['Date_Decision'] = pd.to_datetime(cpts['Date_Decision'])
cpts['Date_Received'] = pd.to_datetime(cpts['Date_Received'])

#### Quality Check
This code is meant to run on 1 year of referrals data.  This QC check ensures that the sql query above pulled the correct amount of data

In [ ]:
## Ensure that the max date from the data is recent, within the last three months
if (pd.datetime.now() - cpts['Date_Received'].max()) < timedelta(90):
    print("Data is near current and runs through {}".format(cpts['Date_Received'].max()))
else:
    print("Please update the referrals.sql file to pull more current data.")

In [ ]:
if (cpts['Date_Received'].max() - cpts['Date_Received'].min()) > timedelta(360):
    if (cpts['Date_Received'].max() - cpts['Date_Received'].min()) < timedelta(370):
        print("Referrals data contains 1 year of data, looks good.")
    else:
        print("Please update the referrals.sql file to pull an entire year of data.")
else:
    print("Please update the referrals.sql file to pull an entire year of data.")

In [ ]:
expected_reasons = ['Pend EPL', 'Pend Specialty', 'Low Volume', np.nan, 'Heavy Cap Vol',
       'Medical Necessity Review', 'Needs Review', 'Override Pend',
       'Upcoding', 'Cost Containment', 'Overutilization Concern',
       'Inappropriate Location', 'RNL', 'Unclassified Drug',
       'Sub-Spec Addition']

In [ ]:
current_reasons = list(clinical_decisions_current['Reason'].unique())

In [ ]:
for rsn in current_reasons:
    if rsn not in expected_reasons:
        print("{} is not in the list of expected reasons.".format(rsn))

In [ ]:
## Some specialties need to be broken into Senior and non-commercial 
## such that we can auto-approve for specific lines of business
new_lob = {'COMMERCIAL': '_not_sen',
               'SENIOR': '_senior',
          'MEDI-CAL': '_not_sen'}
cpts['LOB'] = cpts['LOB'].replace(new_lob)

In [ ]:
## Define a list of specialties that will be broken out into LOB for the purposes of AA
specs_w_lob_distinct = ['RADIOLOGY']

In [ ]:
## Loop through specialties that should be broken by LOB and update the specialty column
for spec in list_o_specs:
    if spec in specs_w_lob_distinct:
        cpts['Specialty'] = np.where(cpts['Specialty']==spec, cpts['Specialty']+cpts['LOB'], cpts['Specialty'])
    

In [ ]:
# Now that some specialties are broken into senior and non-senior, need to recreate the list_o_specs
list_o_specs = cpts['Specialty'].unique().tolist()

In [ ]:
## Append specialties manually that only appear in CCT data
list_o_specs.append('HIV/AIDS SPECIALIST')

In [ ]:
## flag retro statuses with 1 and 0 so they can be removed 
retro_conditions = [
 (cpts['status_name'] == 'APPROVED - RETRO REVIEW') |
 (cpts['status_name'] == 'DENIED - RETRO REVIEW') |
 (cpts['status_name'] == 'APPROVED - COB RETRO') |
 (cpts['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

choices = [1]
cpts['is_retro'] = np.select(retro_conditions, choices, default=0)

## remove retros from list and drop 'is_retro' as it is no longer needed
cpts = cpts[cpts['is_retro']==0]
cpts.drop(columns='is_retro', inplace=True)

In [ ]:
## Claims data doesn't come with UNITS & we need to count the number of times a cpt code appears
## Here we re-write UNITS to 1.
cpts['UNITS'] = 1

In [ ]:
## Create an auto_approve flag
cpts['is_autoapp'] = np.where(cpts['status_name']=='APPROVED - AUTO', 1, 0)

In [ ]:
## If PPL field is null, assume the referral was not PPL
cpts.PPL.fillna("N", inplace=True)

In [ ]:
## Create a PPL flag
cpts['is_PPL'] = np.where(cpts['PPL']=='Y', 1, 0)

In [ ]:
## define list of status that should be considered denials
den_conditions = [
 (cpts['status_name'] == 'DENIED - CM') |
 (cpts['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (cpts['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (cpts['status_name'] == 'DENIED - APPEAL') |
 (cpts['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (cpts['status_name'] == 'DENIED - TRANSPLANT') |
 (cpts['status_name'] == 'DENIED - MD') |
 (cpts['status_name'] == 'DENIED - CM/MD') |
 (cpts['status_name'] == 'DENIED - REDIRECT OSVN') |
 (cpts['status_name'] == 'DENIED - TICKLER')
  ]

## Create a denial flag
choices = [1]
cpts['is_den'] = np.select(den_conditions, choices, default=0)

In [ ]:
## Create a "approved" flag
cpts['is_app'] = np.where(cpts['status_cat']=='APPROVED', 1, 0)

## Create claims_sum

In [ ]:
## Some Specialty/CPT code combos appear in referrals data but not in claims data. 
## In those cases, we look in the claims data across all specialties to find an average
## Cost to be applied for that CPT Code.
claims_sum = claims.groupby(['CPT_Code'], as_index=False).agg({'avg_hcp_cost': 'mean'})

## Create cpts_manual

In [ ]:
## This is a cpt_code level list of all manually reviewed referrals

In [ ]:
cpts_manual = cpts[cpts['is_autoapp']==0]

In [ ]:
## Enter the total GA for the department
dept_ga = 6500000

In [ ]:
## Calculate Cost per manually reviewed CPT code

ga_cpt = dept_ga / cpts[cpts['is_autoapp']==0].shape[0]

In [ ]:
## find the count of manually reviewed cpt codes from each specialty, cpt_code combo 
cpts_manual = cpts_manual.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS' : 'count'
})

In [ ]:
## To differentiate the count of all units from manual units as we use both in a single
## file later
cpts_manual.rename(index=str, columns={'UNITS': 'UNITS_man'}, inplace=True)

In [ ]:
## Calculate the total cost of review any Specialty/cpt_code pair.
cpts_manual['cost_to_review'] = cpts_manual['UNITS_man']*ga_cpt

#### Quality Check
Check to ensure that sum of cost to review column is similar to dept_ga

In [ ]:
if (cpts_manual['cost_to_review'].sum() > dept_ga*.95) & (cpts_manual['cost_to_review'].sum() < dept_ga*1.05):
    print('Quality Check: Sum of Cost to Review is similar to department G&A.')
else:
    print('Sum of Cost to Review and Department G&A are more that 5% different, please check.')

## Create cptssum

In [ ]:
## find the count, auto-approval rate, and denial rate from each specialty, cpt_code pair
cptssum = cpts.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_den': 'mean',
})

In [ ]:
## Merge the cost to review (from CPT_manual) into cptssum
cpts_w_claims0 = pd.merge(cptssum, cpts_manual, on=['Specialty', 'CPT_Code', 'is_PPL'], how='left')

In [ ]:
## Merge average cost of a cpt code (from claims) into the referrals data
cpts_w_claims1 = pd.merge(cpts_w_claims0, claims, on=['Specialty', 'CPT_Code'], how='left')

In [ ]:
## For spec/cpt combos that don't have claims data associated, 
## use the average of that cpt across specialties
## NOTE: the average is not weighted, i.e. each specialties's average contributes equally to
## the applied average.
cpts_w_claims_fin = pd.merge(cpts_w_claims1, claims_sum, on='CPT_Code', how='left')

In [ ]:
## If there is no average cost from claims at the spec/cpt pair level, fill it with the 
## average cost for the cpt ACROSS ALL SPECIALTIES
cpts_w_claims_fin['avg_hcp_cost_x'] = np.where(cpts_w_claims_fin['avg_hcp_cost_x'].isnull(), 
                                             cpts_w_claims_fin['avg_hcp_cost_y'],
                                             cpts_w_claims_fin['avg_hcp_cost_x'])

In [ ]:
## Drop unnecessary columns
cpts_w_claims_fin.drop(columns=['avg_hcp_cost_y', 'sd_hcp_cost'], inplace=True)

In [ ]:
## Rename "_x" to the normal name - 'avg_hcp_cost'
cpts_w_claims_fin.rename(index=str, columns={'avg_hcp_cost_x': 'avg_hcp_cost'}, inplace=True)

In [ ]:
## To ensure calculations are defined, replace nulls with 0.
cpts_w_claims_fin['UNITS_man'] = np.where(cpts_w_claims_fin['UNITS_man'].isnull(), 0, cpts_w_claims_fin['UNITS_man'])

## Calculate ROI

For each spec/cpt code pair, we want to comapre the cost of reviewing the pair with the sum of dollars denied through that review. This is the "ROI" of reviewing. In cases where sum of denied dollars is greater than the cost of review, we recommend NOT auto-approving and continue to review. In cases where sum of denied dollars is less than the cost of review then we recommend auto-approving it. 

In [ ]:
## Calculate the total dollars denied for a pair
cpts_w_claims_fin['sum_cost_denied'] = cpts_w_claims_fin['is_den']*cpts_w_claims_fin['UNITS']*cpts_w_claims_fin['avg_hcp_cost']

In [ ]:
## Calc ROI for a pair
cpts_w_claims_fin['ROI'] = cpts_w_claims_fin['sum_cost_denied']/cpts_w_claims_fin['cost_to_review']

In [ ]:
## For groups where we don't know the average cost from 2018, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 100 so they are NOT included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['avg_hcp_cost'].isnull(), 100, cpts_w_claims_fin['ROI'])

In [ ]:
## For groups that were auto-approved at 100%, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 0 so they are included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['ROI'].isnull(), 0.01, cpts_w_claims_fin['ROI'])
        

In [ ]:
## Create a flag based on ROI indicated whether the analytics recommend a pair to be auto-approved
cpts_w_claims_fin['fin_aa_rec'] = np.where(cpts_w_claims_fin['ROI']<1, 1, 0)

## Create Dictionaries

In [ ]:
## Model Additional Auto Approvals when different thresholds are set.
## Approach: use "given" threshold to determine which CPT codes are "auto-approve"-able for each specialty
##  - For loop through referrals, return 1 if all CPT codes are on "auto-approve"-able list, else 0 

In [ ]:
cpts_w_claims_fin.reset_index(drop=True, inplace=True)

In [ ]:
## A function that uses cpt_w_claims_fin to generate a dictionary codes to auto-approve.
def collect_clinical_decisions(specialty_cpt, list_to_change, spec_list, rsn, new_status=0):
    decision_sources = cpts_w_claims_fin['dec_source'].to_list()
    overrule_reasons = cpts_w_claims_fin['overrule_rsn'].to_list()
    final_decisions = cpts_w_claims_fin['final_decision'].to_list()
    for index, row in specialty_cpt.iterrows():
        if row['Specialty'] in spec_list:
            if row['CPT_Code'] in list_to_change:
                decision_sources[index] = 'clinical'
                overrule_reasons[index] = rsn
                final_decisions[index] = new_status
    specialty_cpt['dec_source'] = decision_sources
    specialty_cpt['overrule_rsn'] = overrule_reasons
    specialty_cpt['final_decision'] = final_decisions
    return specialty_cpt
    

In [ ]:
## Create new column in cpt_w_claims_fin that will store decision source.
cpts_w_claims_fin['dec_source'] = 'DBA'

In [ ]:
## Create new column in cpt_w_claims_fin that will store clinical overrule_rsn
cpts_w_claims_fin['overrule_rsn'] = np.nan

In [ ]:
## Create new column in cpt_w_claims_fin that will store final decision
cpts_w_claims_fin['final_decision'] = cpts_w_claims_fin['fin_aa_rec']

### Overlay Current Clinical Decisions to "override" ROI decisions

So far in the code we've ingests claims and referrals and for each Spec & CPT code combo we've calculated the ROI of reviewing that combo. In cases where the ROI is above one, we put a temporary decision of pend, and when the ROI is low (below 1) we put a temporary decision of auto-approve.  However, clinical decisions can overturn these ROI decisions. Below we:
1. overlay clinical decisions from the AADictionary_Master file
2. As a safeguard, we have several portions of code to ensure some of the more broad clinical decisions are in place.  These portions should be in line with AADictionary_Master file

#### Overlay current clinical decisions "on top" of ROI decisions

In [ ]:
def add_clinical_decisions(cpts_w_claims_fin, clinical_decisions, drop_columns = ['Decision', 'Notes', 'overturned']):
    cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])
    cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Reason'], cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])
    cpts_w_claims_fin.drop(drop_columns, axis=1, inplace=True)
    return cpts_w_claims_fin

In [ ]:
cpts_w_claims_fin = add_clinical_decisions(cpts_w_claims_fin, clinical_decisions_current, ['Decision', 'Reason', 'Decision_Date',
                                                                                           'Decision_Source', 'Decision_Maker'])

#### Soft Pend Specialties categorically

In [ ]:
## Specialties for which the clinical team prefers to ignore the "decision by analysis" recommendation and instead
## choose a few codes to AA, but mostly the specialty will pend
## To ensure that no referrals are auto-approved from it
specs_that_should_pend = ['ALLERGY/IMMUNOLOGY',
 'PODIATRY',
 'PHYSICAL THERAPY/REHAB',
 'FACILITY SERVICES',
 'GENETICS',
 'INTERVENTIONAL RADIOLOGY',
 'LABORATROY',
 'ONCOLOGY - GYN',
 'SURGERY - CARDIAC',
 'SURGERY - MAXILLOFACIAL ORAL',
 'SURGERY - PLASTIC/RECONSTRUCT']

In [ ]:
for spec in specs_that_should_pend:
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Specialty'] == spec, 
                                    'Soft Pend Specialty', cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    'clinical', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    0, cpts_w_claims_fin['final_decision'])

#### Pend codes with fewer than 30 in the past year

In [ ]:
## Pend codes that have volume < 30
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'low volume', cpts_w_claims_fin['overrule_rsn'])

In [ ]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'rule', cpts_w_claims_fin['dec_source'])

In [ ]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    0, cpts_w_claims_fin['final_decision'])

#### Hard Pend Specialties categorically

In [ ]:
## Specialties that the clinical team wants to pend categorically

specs_that_should_pend_hard = ['ACUPUNCTURE', 'ADDICTION MEDICINE', 'ANESTHESIOLOGY',
                         'BEHAVIORAL HEALTH', 'CHIROPRACTIC', 'DENTIST', 'DME MAINTENANCE',
                         'LICENSED CLIN SOCIAL WORKER', 'MFCC (THERAPIST)', 'NON-CONTRACT UNKWN BILL AREA',
                         'NURSE PRACTITIONER', 'NURSING FACILITY - OTHER', 'OPTICIAN',
                        'PEDS-DEVELOPMENTAL BEHAVIORAL', 'PSYCHIATRY', 'PSYCHOLOGY', 'SENIOR WELLNESS VISIT',
                         'SPORTS MEDICINE', 'SURGERY - ORAL', 'AMBULATORY SURGICAL CENTER', 'AMBULANCE',
                         'CUSTODIAL CARE', 'HOME HEALTH', 'INFERTILITY', 'OPTOMETRY', 'PALLIATIVE CARE', 'SLEEP STUDY',
                         'SNF - FAC', 'OCCUPATIONAL THERAPY', 'NUCLEAR MEDICINE',
                         'PATHOLOGY', 'PHARMACY', 'SURGERY - HAND']

In [ ]:
for spec in specs_that_should_pend_hard:
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Specialty'] == spec, 
                                    'Hard Pend Specialty', cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    'rule', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    0, cpts_w_claims_fin['final_decision'])

#### Pend all EPL

In [ ]:
## Pend EPL
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'Pend EPL', cpts_w_claims_fin['overrule_rsn'])

In [ ]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'rule', cpts_w_claims_fin['dec_source'])

In [ ]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    0, cpts_w_claims_fin['final_decision'])

## Create AA dictionaries

In [ ]:
## Function that gathers CPT codes that should be pended and stores them in a dictionary.
def create_dict_of_CPT_codes_v3(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['final_decision'] == 1:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict

In [ ]:
## Ensure that the dictionary for EPL is empty, meaning none will auto-approve
spec_dict_EPL = {k: [] for k in list_o_specs}

In [ ]:
spec_dict_PPL = create_dict_of_CPT_codes_v3(cpts_w_claims_fin, list_o_specs, PPL=1)

### Generate Predictions of AA Rate

In [ ]:
## Assigns an auto-approve (1) or pend (0) status to each specialty / code group
def assign_status(codes, spec_dict_PPL, spec_dict_EPL, list_of_types_pend):
    status = list(np.zeros(codes.shape[0]))
    specs_list = codes['Specialty'].unique().tolist()
    print(len(status))
    print(codes.shape[0])
    for spec in specs_list:
        if spec not in spec_dict_PPL:
            spec_dict_PPL[spec] = []
    for spec in specs_list:
        if spec not in spec_dict_EPL:
            spec_dict_EPL[spec] = []
    for index, row in codes.iterrows():
        if row['is_PPL'] == 1:
            if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
                if row['ref_type'] not in list_of_types_pend:
                    status[index] = 1
        ##else:
          ##  if row['CPT_Code'] in spec_dict_EPL[row['Specialty']]:
            ##    status[index] = 0
    return status

In [ ]:
cpts.reset_index(drop=True, inplace=True)

In [ ]:
list_o_types_to_pend = ['INPT ADM', 'EMERGENCY ROOM', 'DAY SURG', 'OOA INPT', 
                        'SKILLED NURSING', 'OBSERVATION', 'OB OBSERVATION']

In [ ]:
auto_approve = assign_status(cpts, spec_dict_PPL, 
                             spec_dict_EPL, list_o_types_to_pend)

In [ ]:
cpts['auto_approvable'] = auto_approve

In [ ]:
refs_results = cpts.groupby(['HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({'auto_approvable': 'mean',
                                                                                 'is_den': 'mean'})

In [ ]:
refs_results['aa-yn'] = np.where(refs_results['auto_approvable']==1, 1, 0)

In [ ]:
refs_results.drop('auto_approvable', axis=1, inplace=True)

In [ ]:
refs_results.pivot_table('HCP_CONNECT_AUTH_NUMBER', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

In [ ]:
refs_results.drop('is_den', axis=1, inplace=True)

In [ ]:
cpts = cpts.merge(refs_results, how='left', on=['HCP_CONNECT_AUTH_NUMBER'])

In [ ]:
print('Predicted auto approval rate is {:f}, including a 4% haircut for inability to model facilities'.format(refs_results['aa-yn'].mean()-.04))

## Construct Specialty Summary

In [ ]:
refs_new_head = cpts.groupby(['Specialty', 'ref_type', 'HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_PPL': 'mean',
    'is_app': 'mean',
    'is_den': 'mean',
    'aa-yn': 'mean'
})

In [ ]:
refs_new_head.pivot_table(values=['is_autoapp', 'aa-yn'], index='Specialty', aggfunc=['count', 'mean']).to_csv('../Data/Outputs/spec_rates.csv',
                                                                                                              sep='|')

In [ ]:
specialty_summary = pd.read_csv('../Data/Outputs/spec_rates.csv', sep='|', skiprows=3, names=['Specialty',
                                                                                             'ref_vol', 'ref_vol2',
                                                                                             'new_rate', 'old_rate'])

In [ ]:
specialty_summary.drop('ref_vol2', axis=1, inplace=True)

In [ ]:
## Get counts of codes for each specialty

In [ ]:
def spec_code_counts(spec_dict, spec_summary):
    counts = []
    for index, row in spec_summary.iterrows():
        counts.append(len(spec_dict[row['Specialty']]))
    return counts

In [ ]:
PPL_counts = spec_code_counts(spec_dict_PPL, specialty_summary)

In [ ]:
specialty_summary['num_codes']=PPL_counts

In [ ]:
## get 'denied now approved'

In [ ]:
refs_new_head['denied_now_aa'] = refs_new_head['is_den'] * refs_new_head['aa-yn']

In [ ]:
refs_new_head.pivot_table('denied_now_aa', index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/spec_den_now_aa.csv',
                                                                                                      sep='|')

In [ ]:
ss_deny_now_aa = pd.read_csv('../Data/Outputs/spec_den_now_aa.csv', sep='|', skiprows=2, names=['Specialty', 'deny_now_aa'])

In [ ]:
specialty_summary = specialty_summary.merge(ss_deny_now_aa, how='inner', on='Specialty')

In [ ]:
## Prep cpts_w_claims_fin to merge into cpts

In [ ]:
cpts_w_claims_fin_select = cpts_w_claims_fin.drop(['UNITS', 'is_autoapp', 'is_den',
       'UNITS_man', 'cost_to_review', 'cnt_hcp_cost',
       'sum_cost_denied'], axis=1)

In [ ]:
cpts_w_fin = pd.merge(cpts, cpts_w_claims_fin_select, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [ ]:
cpts_w_fin['dollars_denied_now_app'] = cpts_w_fin['is_den']*cpts_w_fin['aa-yn']*cpts_w_fin['avg_hcp_cost']

In [ ]:
cpts_w_fin['cost_to_review_bene'] = (1-cpts_w_fin['is_autoapp'])*cpts_w_fin['aa-yn']*ga_cpt

In [ ]:
cpts_w_fin['cost_to_review_loss'] = cpts_w_fin['is_autoapp']*(1-cpts_w_fin['aa-yn'])*ga_cpt

In [ ]:
cpts_w_fin['new_denial_bene_est'] = cpts_w_fin['is_autoapp']*(1-cpts_w_fin['aa-yn'])*ga_cpt*cpts_w_fin['ROI']*cpts_w_fin['is_PPL']

In [ ]:
cpts_w_fin.pivot_table(values=['dollars_denied_now_app', 'cost_to_review_bene', 'cost_to_review_loss',
                                  'new_denial_bene_est'], index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [ ]:
specialty_fins = pd.read_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [ ]:
specialty_summary = specialty_summary.merge(specialty_fins, how='inner', on='Specialty')

In [ ]:
specialty_summary['net_benefit'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] - specialty_summary['dollars_denied_now_app']

In [ ]:
specialty_summary['net_benefit_w_new_den'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] -specialty_summary['dollars_denied_now_app'] + specialty_summary['new_denial_bene_est']

In [ ]:
# Give the filename you wish to save the file to
spec_summary_filename = '../Data/Outputs/specialty_summary_fin.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_summary_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    specialty_summary.to_excel(spec_summary_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'volume', 'new rate', 'old_rate', '# codes in dictionary', 'false positives',
                                 'cost to review, benefit', 'cost to review, loss', 'dollars denied, now approved',
                                 'new denials, benefit (est)', 'new benefit', 'net benefit, est'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [ ]:
specialty_summary

In [ ]:
cpts_w_claims_fin['overrule_rsn'].unique()

## Code Detail List

In [ ]:
spec_cpt_w_projected_aa_rate = cpts.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'aa-yn': 'mean'
})

In [ ]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(spec_cpt_w_projected_aa_rate, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [ ]:
cpts['UNITS'].sum()

In [ ]:
# Give the filename you wish to save the file to
spec_cpt_w_projected_aa_rate_fin_filename = '../Data/Outputs/spec_cpt_w_projected_aa_rate.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_cpt_w_projected_aa_rate_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    spec_cpt_w_projected_aa_rate.to_excel(spec_cpt_w_projected_aa_rate_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'new rate'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [ ]:
cpts.pivot_table('CPT_Code', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

In [ ]:
cpts_w_claims_fin.drop(['cnt_hcp_cost'], axis=1, inplace=True)

In [ ]:
cpt_desc = pd.read_csv('../Data/cpt_desc_raw.csv', sep='|')

In [ ]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(cpt_desc, how='inner', on = 'CPT_Code')

In [ ]:
cpts_w_claims_fin.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
cpts_w_claims_fin[cpts_w_claims_fin['Specialty']=='CARDIOLOGY']

In [ ]:
# Give the filename you wish to save the file to
cpts_w_claims_fin_filename = '../Data/Outputs/AADictionary_New.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(cpts_w_claims_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    cpts_w_claims_fin.to_excel(cpts_w_claims_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'volume', 'old rate', 'denial rate',
       'volume, reviewed', 'cost to review', 'avg. cost', 'denied dollars', 'ROI',
       'decision by ROI', 'decision source', 'clinical reason', 'final decision', 'new rate',
       'CPT desc'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')